In [1]:
import h2o4gpu
import h2o4gpu.util.import_data as io
import h2o4gpu.util.metrics as metrics

In [2]:
"""
Import Data for H2O GPU Edition

This function will read in data and prepare it for H2O4GPU's GLM solver

Parameters
----------
data_path : str
             A path to a dataset (The dataset needs to be all numeric)
use_pandas : bool
              Indicate if Pandas should be used to parse
intercept : bool
              Indicate if intercept term is needed
valid_fraction : float
                  Percentage of dataset reserved for a validation set
classification : bool
                  Classification problem?
Returns
-------
If valid_fraction > 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    valid_x: numpy array of valid input variables
    valid_y: numpy array of valid y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
If valid_fraction == 0 it will return the following:
    train_x: numpy array of train input variables
    train_y: numpy array of y variable
    family : string that would either be "logistic" if classification is set to True, otherwise "elasticnet"
"""

train_x,train_y,valid_x,valid_y,family=io.import_data(data_path="../../../data/credit.feather", 
                                                        use_pandas=False, 
                                                        intercept=True,
                                                        valid_fraction=0.2,
                                                        classification=True)

Reading Data with Feather
(23999, 25)
Original m=23999 n=24
Size of Train rows=19200 & valid rows=4799
Size of Train cols=24 valid cols=24
Size of Train cols=25 & valid cols=25 after adding intercept column


In [3]:
"""
Set up instance of H2O4GPU's GLM solver with default parameters

Need to pass in `family` to indicate problem type to solve
"""
print("Setting up solver")
model = h2o4gpu.GLM(family=family)

Setting up solver


In [4]:
"""
Fit GLM Solver
"""
print("Solving")
%time model.fit(train_x, train_y)
print("Done Solving")

Solving
CPU times: user 17.2 s, sys: 2.86 s, total: 20 s
Wall time: 11.1 s
Done Solving


In [5]:
"""
Obtain train error. Logloss for classification and RMSE(Root Mean Squared Error) for regression.
"""
error_train = model.error_best
if family == "logistic":
    print("logloss_train per alpha value")
else:
    print("rmse_train per alpha value")
print(error_train)

logloss_train per alpha value
[[ 0.47696245  0.47602504 -1.        ]
 [ 0.47703174  0.47588971 -1.        ]
 [ 0.47718677  0.47587493 -1.        ]
 [ 0.47715604  0.47587681 -1.        ]
 [ 0.47735882  0.47599381 -1.        ]]


In [7]:
"""
Make predictions on validation set
"""
preds = model.predict(valid_x, valid_y)
print(preds)

[[ 0.75720805  0.14413147  0.14343223 ...,  0.2697956   0.24582249
   0.17748925]
 [ 0.76252538  0.14352286  0.14560172 ...,  0.26030022  0.24178487
   0.17574789]
 [ 0.76570785  0.1461037   0.15109476 ...,  0.25633189  0.24425988
   0.18305215]
 [ 0.7688002   0.14296913  0.15202616 ...,  0.25164178  0.23972832
   0.18067253]
 [ 0.76751584  0.14442751  0.15594704 ...,  0.24513918  0.23969832
   0.18374185]]


In [8]:
"""
Get logloss or rmse for validation set per alpha
"""
for i in range(model.n_alphas):
    if family == "logistic":
        print("Logloss for alpha = ",model.alphas_best[i])
        print(metrics.log_loss(valid_y, preds[i]))
    else:
        print("RMSE for alpha = ",model.alphas_best[i])
        print(metrics.rmse(valid_y,preds[i]))

Logloss for alpha =  [ 0.]
0.438944581836
Logloss for alpha =  [ 0.25]
0.438956755434
Logloss for alpha =  [ 0.5]
0.439326538957
Logloss for alpha =  [ 0.75]
0.439263982683
Logloss for alpha =  [ 1.]
0.439506193298
